#

# Car Data Monitoring Experiments

This notebook aims at experimenting with the data obtained from the car data recorder.

In [1]:
import polars as pl
import plotly.express as px

In [ ]:
df = pl.read_csv(
    "/Users/leopnt/Downloads/0bbce73/gps/2025-02-05T074234Z.csv",
    try_parse_dates=True,
)

df

## Time stability

First, let's explore the most important reference variable: datetime and millis. Are they stable?

In the firmware code, there are 3 clocks:
- The internal ESP32 clock, which is independant of external sources but may drift against true time
- The GPS clock, which is synced with satellite clock information internaly, every time it receives a signal.
- A software wrapper, that I coded on top of a Clock library, is synced when the GPS gives updates. It uses the internal ESP32 clock to add millisecond precision.

To ensure that all information is available, I log every event with the synced software clock's timestamp. But I also add the internal ESP32 clock as a backup measure.
Let's see if this clocks are in sync.

In [ ]:
fig = px.scatter(data_frame=df, x='datetime', y='millis', title="Millis() vs time")
fig.show()

Looking at the two timestamps from a trip recording, we can see that
- A few minutes of data are missing. This is due to GPS Signal loss. Not a big deal here, since 1 minute of data will probably be filled thanks to dead reckoning with other sensors and CAN Bus data.
- It looks like there is no significative drift between the two clocks since we have a clean straight line. This is good.

Now let's take a look at the delta time, which will give us a more precise look. There should be the same time variations for each clock.

In [4]:
df = df.with_columns(
    delta_time=pl.col('datetime').diff(),
    delta_millis=pl.duration(milliseconds=(pl.col('millis').diff()))
)

In [ ]:
fig = px.ecdf(
    df.with_columns(
        pl.col("delta_time").cast(pl.Float32) / 1000000,
        pl.col("delta_millis").cast(pl.Float32) / 1000000
    ), x=['delta_time', 'delta_millis'],
    title='Time variations of the two clocks')
fig.update_layout(xaxis_range=[0.0, 2.0])
fig.show()

Zooming at the data between 0s and 2s, we can see that the clocks are fairly stable around 1s. I don't know why the internal ESP32 clock is not fully aligned with the software one. I believed it is induced by software processing or GPS synchronization in between.

I can see that the software clock is much more stable than I would have anticipated.
Outliers for the software clocks are most probably due to GPS loss as seen with previous plots.